# Generative netværk

Recurrent Neural Networks (RNNs) og deres gatede cellevarianter såsom Long Short Term Memory Cells (LSTMs) og Gated Recurrent Units (GRUs) gav en mekanisme til sproglig modellering, dvs. de kan lære ordstilling og give forudsigelser for det næste ord i en sekvens. Dette gør det muligt for os at bruge RNNs til **generative opgaver**, såsom almindelig tekstgenerering, maskinoversættelse og endda billedtekstning.

I RNN-arkitekturen, som vi diskuterede i den forrige enhed, producerede hver RNN-enhed den næste skjulte tilstand som output. Men vi kan også tilføje et andet output til hver rekurrent enhed, hvilket giver os mulighed for at generere en **sekvens** (som har samme længde som den oprindelige sekvens). Desuden kan vi bruge RNN-enheder, der ikke modtager input ved hvert trin, men blot tager en initial tilstandsvektor og derefter producerer en sekvens af outputs.

I denne notebook vil vi fokusere på simple generative modeller, der hjælper os med at generere tekst. For enkelhedens skyld lad os bygge et **karakter-niveau netværk**, som genererer tekst bogstav for bogstav. Under træning skal vi tage en tekstkorpus og opdele den i bogstavsekvenser.


In [1]:
import tensorflow as tf
from tensorflow import keras
import tensorflow_datasets as tfds
import numpy as np

ds_train, ds_test = tfds.load('ag_news_subset').values()

## Opbygning af tegnordforråd

For at opbygge et generativt netværk på tegnniveau skal vi dele tekst op i individuelle tegn i stedet for ord. `TextVectorization`-laget, som vi tidligere har brugt, kan ikke gøre dette, så vi har to muligheder:

* Indlæse tekst manuelt og udføre tokenisering 'i hånden', som vist i [dette officielle Keras-eksempel](https://keras.io/examples/generative/lstm_character_level_text_generation/)
* Bruge `Tokenizer`-klassen til tokenisering på tegnniveau.

Vi vælger den anden mulighed. `Tokenizer` kan også bruges til at tokenisere i ord, så det bør være nemt at skifte fra tokenisering på tegnniveau til ordniveau.

For at udføre tokenisering på tegnniveau skal vi angive parameteren `char_level=True`:


In [2]:
def extract_text(x):
    return x['title']+' '+x['description']

def tupelize(x):
    return (extract_text(x),x['label'])

tokenizer = keras.preprocessing.text.Tokenizer(char_level=True,lower=False)
tokenizer.fit_on_texts([x['title'].numpy().decode('utf-8') for x in ds_train])

Vi ønsker også at bruge en speciel token til at angive **slut på sekvens**, som vi vil kalde `<eos>`. Lad os tilføje den manuelt til ordforrådet:


In [3]:
eos_token = len(tokenizer.word_index)+1
tokenizer.word_index['<eos>'] = eos_token

vocab_size = eos_token + 1

Nu, for at kode tekst til sekvenser af tal, kan vi bruge:


In [4]:
tokenizer.texts_to_sequences(['Hello, world!'])

[[48, 2, 10, 10, 5, 44, 1, 25, 5, 8, 10, 13, 78]]

## Træning af en generativ RNN til at generere titler

Måden, vi vil træne RNN til at generere nyhedstitler, er som følger. Ved hvert trin tager vi en titel, som bliver fodret ind i en RNN, og for hvert inputtegn beder vi netværket om at generere det næste outputtegn:

![Billede, der viser et eksempel på RNN-generering af ordet 'HELLO'.](../../../../../translated_images/da/rnn-generate.56c54afb52f9781d.webp)

For det sidste tegn i vores sekvens beder vi netværket om at generere `<eos>`-token.

Den største forskel mellem den generative RNN, vi bruger her, er, at vi tager output fra hvert trin i RNN'en og ikke kun fra den sidste celle. Dette kan opnås ved at angive parameteren `return_sequences` til RNN-cellen.

Således vil input til netværket under træningen være en sekvens af kodede tegn af en vis længde, og output vil være en sekvens af samme længde, men forskudt med ét element og afsluttet med `<eos>`. Minibatch vil bestå af flere sådanne sekvenser, og vi vil være nødt til at bruge **padding** for at justere alle sekvenser.

Lad os oprette funktioner, der vil transformere datasættet for os. Fordi vi ønsker at padde sekvenser på minibatch-niveau, vil vi først batch'e datasættet ved at kalde `.batch()`, og derefter `map` det for at udføre transformationen. Så transformationsfunktionen vil tage en hel minibatch som parameter:


In [5]:
def title_batch(x):
    x = [t.numpy().decode('utf-8') for t in x]
    z = tokenizer.texts_to_sequences(x)
    z = tf.keras.preprocessing.sequence.pad_sequences(z)
    return tf.one_hot(z,vocab_size), tf.one_hot(tf.concat([z[:,1:],tf.constant(eos_token,shape=(len(z),1))],axis=1),vocab_size)

Et par vigtige ting, vi gør her:
* Vi starter med at udtrække den faktiske tekst fra string-tensoren
* `text_to_sequences` konverterer listen af strenge til en liste af heltals-tensorer
* `pad_sequences` udfylder disse tensorer til deres maksimale længde
* Til sidst one-hot-enkoder vi alle tegnene og udfører også forskydning og tilføjelse af `<eos>`. Vi vil snart se, hvorfor vi har brug for one-hot-enkodede tegn

Dog er denne funktion **Pythonisk**, dvs. den kan ikke automatisk oversættes til Tensorflow's beregningsgraf. Vi vil få fejl, hvis vi forsøger at bruge denne funktion direkte i `Dataset.map`-funktionen. Vi er nødt til at omslutte dette Pythoniske kald ved at bruge `py_function`-wrapperen:


In [6]:
def title_batch_fn(x):
    x = x['title']
    a,b = tf.py_function(title_batch,inp=[x],Tout=(tf.float32,tf.float32))
    return a,b

> **Note**: Det kan virke lidt for komplekst at skelne mellem Pythoniske og Tensorflow-transformationsfunktioner, og du undrer dig måske over, hvorfor vi ikke transformerer datasættet ved hjælp af standard Python-funktioner, før vi sender det til `fit`. Selvom dette bestemt kan lade sig gøre, har brugen af `Dataset.map` en stor fordel, fordi datatransformationspipeline udføres ved hjælp af Tensorflows beregningsgraf, som drager fordel af GPU-beregninger og minimerer behovet for at overføre data mellem CPU/GPU.

Nu kan vi bygge vores generatornetværk og begynde træningen. Det kan baseres på enhver rekurrent celle, som vi diskuterede i den foregående enhed (simpel, LSTM eller GRU). I vores eksempel vil vi bruge LSTM.

Da netværket tager tegn som input, og ordforrådsstørrelsen er ret lille, har vi ikke brug for et embedding-lag; one-hot-kodet input kan direkte sendes ind i LSTM-cellen. Outputlaget vil være en `Dense` klassifikator, der konverterer LSTM-output til one-hot-kodede token-numre.

Derudover, da vi arbejder med sekvenser af variabel længde, kan vi bruge `Masking`-laget til at skabe en maske, der ignorerer den polstrede del af strengen. Dette er ikke strengt nødvendigt, da vi ikke er særligt interesserede i alt, der går ud over `<eos>`-tokenet, men vi vil bruge det for at få noget erfaring med denne lagtype. `input_shape` vil være `(None, vocab_size)`, hvor `None` angiver sekvensen af variabel længde, og outputformen er også `(None, vocab_size)`, som du kan se fra `summary`:


In [7]:
model = keras.models.Sequential([
    keras.layers.Masking(input_shape=(None,vocab_size)),
    keras.layers.LSTM(128,return_sequences=True),
    keras.layers.Dense(vocab_size,activation='softmax')
])

model.summary()
model.compile(loss='categorical_crossentropy')

model.fit(ds_train.batch(8).map(title_batch_fn))

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
masking (Masking)            (None, None, 84)          0         
_________________________________________________________________
lstm (LSTM)                  (None, None, 128)         109056    
_________________________________________________________________
dense (Dense)                (None, None, 84)          10836     
Total params: 119,892
Trainable params: 119,892
Non-trainable params: 0
_________________________________________________________________
15000/15000 [==============================] - 229s 15ms/step - loss: 1.5385


## Generering af output

Nu hvor vi har trænet modellen, vil vi bruge den til at generere noget output. Først og fremmest har vi brug for en måde at dekode tekst repræsenteret ved en sekvens af token-numre. For at gøre dette kunne vi bruge funktionen `tokenizer.sequences_to_texts`; dog fungerer den ikke godt med tokenisering på tegnniveau. Derfor vil vi tage en ordbog af tokens fra tokenizer (kaldet `word_index`), opbygge et omvendt kort og skrive vores egen dekoderingsfunktion:


In [10]:
reverse_map = {val:key for key, val in tokenizer.word_index.items()}

def decode(x):
    return ''.join([reverse_map[t] for t in x])

Nu begynder vi med en streng `start`, koder den til en sekvens `inp`, og derefter kalder vi vores netværk på hvert trin for at udlede det næste tegn.

Outputtet fra netværket `out` er en vektor med `vocab_size` elementer, der repræsenterer sandsynlighederne for hver token, og vi kan finde det mest sandsynlige token-nummer ved at bruge `argmax`. Derefter tilføjer vi dette tegn til den genererede liste af tokens og fortsætter med genereringen. Denne proces med at generere ét tegn gentages `size` gange for at generere det ønskede antal tegn, og vi afslutter tidligt, når `eos_token` optræder.


In [12]:
def generate(model,size=100,start='Today '):
        inp = tokenizer.texts_to_sequences([start])[0]
        chars = inp
        for i in range(size):
            out = model(tf.expand_dims(tf.one_hot(inp,vocab_size),0))[0][-1]
            nc = tf.argmax(out)
            if nc==eos_token:
                break
            chars.append(nc.numpy())
            inp = inp+[nc]
        return decode(chars)
    
generate(model)

'Today #39;s lead to strike for the strike for the strike for the strike (AFP)'

## Udtagning af output under træning

Da vi ikke har nogen nyttige målepunkter som *nøjagtighed*, er den eneste måde, vi kan se, at vores model bliver bedre, ved at **udtage** genererede strenge under træningen. For at gøre dette vil vi bruge **callbacks**, dvs. funktioner, som vi kan give videre til `fit`-funktionen, og som vil blive kaldt periodisk under træningen.


In [13]:
sampling_callback = keras.callbacks.LambdaCallback(
  on_epoch_end = lambda batch, logs: print(generate(model))
)

model.fit(ds_train.batch(8).map(title_batch_fn),callbacks=[sampling_callback],epochs=3)

Epoch 1/3
15000/15000 [==============================] - 226s 15ms/step - loss: 1.2703
Today #39;s a lead in the company for the strike
Epoch 2/3
15000/15000 [==============================] - 227s 15ms/step - loss: 1.2057
Today #39;s the Market Service on Security Start (AP)
Epoch 3/3
15000/15000 [==============================] - 226s 15ms/step - loss: 1.1752
Today #39;s a line on the strike to start for the start


Dette eksempel genererer allerede ret god tekst, men det kan forbedres yderligere på flere måder:

* **Mere tekst**. Vi har kun brugt titler til vores opgave, men du kan eksperimentere med fuld tekst. Husk, at RNN'er ikke er så gode til at håndtere lange sekvenser, så det giver mening enten at opdele dem i kortere sætninger eller altid at træne på en fast sekvenslængde af en foruddefineret værdi `num_chars` (for eksempel 256). Du kan prøve at ændre eksemplet ovenfor til en sådan arkitektur ved at bruge [den officielle Keras-tutorial](https://keras.io/examples/generative/lstm_character_level_text_generation/) som inspiration.

* **Multilags LSTM**. Det giver mening at prøve 2 eller 3 lag af LSTM-celler. Som vi nævnte i den tidligere enhed, udtrækker hvert lag af LSTM visse mønstre fra teksten, og i tilfælde af en generator på tegnniveau kan vi forvente, at det laveste LSTM-lag er ansvarligt for at udtrække stavelser, og de højere lag - for ord og ordkombinationer. Dette kan nemt implementeres ved at sende parameteren for antal lag til LSTM-konstruktøren.

* Du kan også eksperimentere med **GRU-enheder** og se, hvilke der fungerer bedre, samt med **forskellige størrelser på skjulte lag**. Et for stort skjult lag kan resultere i overtilpasning (f.eks. netværket vil lære præcis tekst), og en mindre størrelse kan muligvis ikke producere gode resultater.


## Blød tekstgenerering og temperatur

I den tidligere definition af `generate` valgte vi altid det tegn med den højeste sandsynlighed som det næste tegn i den genererede tekst. Dette resulterede ofte i, at teksten "cyklede" mellem de samme tegnsekvenser igen og igen, som i dette eksempel:
```
today of the second the company and a second the company ...
```

Men hvis vi ser på sandsynlighedsfordelingen for det næste tegn, kan det være, at forskellen mellem de højeste sandsynligheder ikke er særlig stor, f.eks. kan ét tegn have en sandsynlighed på 0,2, mens et andet har 0,19 osv. For eksempel, når vi leder efter det næste tegn i sekvensen '*play*', kan det næste tegn lige så godt være et mellemrum eller **e** (som i ordet *player*).

Dette fører os til konklusionen, at det ikke altid er "retfærdigt" at vælge tegnet med den højeste sandsynlighed, fordi det at vælge det næsthøjeste stadig kan føre til meningsfuld tekst. Det er mere fornuftigt at **udvælge** tegn fra sandsynlighedsfordelingen givet af netværkets output.

Denne udvælgelse kan udføres ved hjælp af funktionen `np.multinomial`, som implementerer den såkaldte **multinomialfordeling**. En funktion, der implementerer denne **bløde** tekstgenerering, er defineret nedenfor:


In [33]:
def generate_soft(model,size=100,start='Today ',temperature=1.0):
        inp = tokenizer.texts_to_sequences([start])[0]
        chars = inp
        for i in range(size):
            out = model(tf.expand_dims(tf.one_hot(inp,vocab_size),0))[0][-1]
            probs = tf.exp(tf.math.log(out)/temperature).numpy().astype(np.float64)
            probs = probs/np.sum(probs)
            nc = np.argmax(np.random.multinomial(1,probs,1))
            if nc==eos_token:
                break
            chars.append(nc)
            inp = inp+[nc]
        return decode(chars)

words = ['Today ','On Sunday ','Moscow, ','President ','Little red riding hood ']
    
for i in [0.3,0.8,1.0,1.3,1.8]:
    print(f"\n--- Temperature = {i}")
    for j in range(5):
        print(generate_soft(model,size=300,start=words[j],temperature=i))


--- Temperature = 0.3
Today #39;s strike #39; to start at the store return
On Sunday PO to Be Data Profit Up (Reuters)
Moscow, SP wins straight to the Microsoft #39;s control of the space start
President olding of the blast start for the strike to pay &lt;b&gt;...&lt;/b&gt;
Little red riding hood ficed to the spam countered in European &lt;b&gt;...&lt;/b&gt;

--- Temperature = 0.8
Today countie strikes ryder missile faces food market blut
On Sunday collores lose-toppy of sale of Bullment in &lt;b&gt;...&lt;/b&gt;
Moscow, IBM Diffeiting in Afghan Software Hotels (Reuters)
President Ol Luster for Profit Peaced Raised (AP)
Little red riding hood dace on depart talks #39; bank up

--- Temperature = 1.0
Today wits House buiting debate fixes #39; supervice stake again
On Sunday arling digital poaching In for level
Moscow, DS Up 7, Top Proble Protest Caprey Mamarian Strike
President teps help of roubler stepted lessabul-Dhalitics (AFP)
Little red riding hood signs on cash in Carter-youb

---

KeyError: 0

Vi har introduceret en ekstra parameter kaldet **temperatur**, som bruges til at angive, hvor strengt vi skal holde os til den højeste sandsynlighed. Hvis temperaturen er 1,0, udfører vi retfærdig multinomial sampling, og når temperaturen går mod uendelig - bliver alle sandsynligheder lige, og vi vælger tilfældigt den næste karakter. I eksemplet nedenfor kan vi observere, at teksten bliver meningsløs, når vi øger temperaturen for meget, og den ligner "cyklet" hårdt-genereret tekst, når den nærmer sig 0.



---

**Ansvarsfraskrivelse**:  
Dette dokument er blevet oversat ved hjælp af AI-oversættelsestjenesten [Co-op Translator](https://github.com/Azure/co-op-translator). Selvom vi bestræber os på nøjagtighed, skal du være opmærksom på, at automatiserede oversættelser kan indeholde fejl eller unøjagtigheder. Det originale dokument på dets oprindelige sprog bør betragtes som den autoritative kilde. For kritisk information anbefales professionel menneskelig oversættelse. Vi påtager os ikke ansvar for eventuelle misforståelser eller fejltolkninger, der opstår som følge af brugen af denne oversættelse.
